In [5]:
# Import library files

import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model

print(tf.__version__)

2.4.1


In [15]:
# Load the dataset
newsize = (200, 200)
X=np.zeros((20579,200,200,3))
files = glob.glob('Images/**/*.jpg', recursive=True)
labels=[]
for i,filename in enumerate(files):
    with Image.open(filename) as im:
        im = im.resize(newsize)
        a=np.array(im)
        if a.shape[2]>3:
            print(filename)
            continue
        X[i]=a
# plt.plot(X[0,:,:,0])

In [16]:
path = 'Images/'
count = []
folders = ([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]) # get all directories
# x=[x.split('-')[1] for x in folders[:20]]
# print(x)
print(folders)
for folder in folders:
    contents = os.listdir(os.path.join(path,folder)) # get list of contents
    count.append(len(contents))
print(count)
count[folders.index('n02105855-Shetland_sheepdog')]=156

['n02097658-silky_terrier', 'n02092002-Scottish_deerhound', 'n02099849-Chesapeake_Bay_retriever', 'n02091244-Ibizan_hound', 'n02095314-wire-haired_fox_terrier', 'n02091831-Saluki', 'n02102318-cocker_spaniel', 'n02104365-schipperke', 'n02090622-borzoi', 'n02113023-Pembroke', 'n02105505-komondor', 'n02093256-Staffordshire_bullterrier', 'n02113799-standard_poodle', 'n02109961-Eskimo_dog', 'n02089973-English_foxhound', 'n02099601-golden_retriever', 'n02095889-Sealyham_terrier', 'n02085782-Japanese_spaniel', 'n02097047-miniature_schnauzer', 'n02110063-malamute', 'n02105162-malinois', 'n02086079-Pekinese', 'n02097130-giant_schnauzer', 'n02113978-Mexican_hairless', 'n02107142-Doberman', 'n02097209-standard_schnauzer', 'n02115913-dhole', 'n02106662-German_shepherd', 'n02106382-Bouvier_des_Flandres', 'n02110185-Siberian_husky', 'n02094258-Norwich_terrier', 'n02093991-Irish_terrier', 'n02094114-Norfolk_terrier', 'n02109525-Saint_Bernard', 'n02093754-Border_terrier', 'n02105251-briard', 'n0210855

In [17]:
# Load labels
Y = np.zeros((20579, 120))
v=0
for i in range(len(count)):
#     Y[v:v+count[i], i] = 1.0
#     v += count[i]
    for j in range(count[i]):
        Y[v,i]=1.0
        v+=1
print(Y[-1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [18]:
l = 3416
# Y = np.zeroes(())
X = X[:l]
Y = Y[:l, :20]
print(X.shape)
print(Y.shape)
print(Y[0])

(3416, 200, 200, 3)
(3416, 20)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
# Split data into train, test, and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=1) # 90:10
X_val, X_test, Y_val, Y_test  = train_test_split(X_val, Y_val, test_size=0.5, random_state=1) # 5:5

# Print shapes
print("Shape of X_train = ",X_train.shape)
print("Shape of Y_train = ",Y_train.shape)
print("Shape of X_val = ",X_val.shape)
print("Shape of Y_val = ",Y_val.shape)
print("Shape of X_test = ",X_test.shape)
print("Shape of Y_test = ",Y_test.shape)

Shape of X_train =  (3074, 200, 200, 3)
Shape of Y_train =  (3074, 20)
Shape of X_val =  (171, 200, 200, 3)
Shape of Y_val =  (171, 20)
Shape of X_test =  (171, 200, 200, 3)
Shape of Y_test =  (171, 20)


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

model=Sequential()
model.add(Conv2D(16, ( 3, 3), padding='same', activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, ( 3, 3), padding='same', activation='relu', input_shape=(200, 200, 16)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(200, 200, 32)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

# model=Sequential()
# model.add(Conv2D(16, (3, 3, 3), padding='same', activation='relu', input_shape=(200, 200, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(32, (3, 3, 16), padding='same', activation='relu', input_shape=(200, 200, 16)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3, 32), padding='same', activation='relu', input_shape=(200, 200, 32)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256))
# model.add(tf.keras.layers.Dense(20, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 128)      

In [25]:
tf.keras.utils.plot_model(
    model, to_file='model.png'
)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [18]:
# model = load_model('model_0.h5')

In [24]:
# Train the model
bs = [1, 16, 32, 64, 128]
# len(X_train)/16
history = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs = 20, batch_size = 16)

Epoch 1/20
193/193 [==============================] - 50s 257ms/step - loss: 26.3617 - categorical_accuracy: 0.0720 - val_loss: 2.9888 - val_categorical_accuracy: 0.1345
Epoch 2/20
193/193 [==============================] - 50s 258ms/step - loss: 2.8424 - categorical_accuracy: 0.1464 - val_loss: 3.0408 - val_categorical_accuracy: 0.0936
Epoch 3/20
193/193 [==============================] - 54s 278ms/step - loss: 2.6570 - categorical_accuracy: 0.1842 - val_loss: 3.0995 - val_categorical_accuracy: 0.1170
Epoch 4/20
193/193 [==============================] - 53s 275ms/step - loss: 2.4637 - categorical_accuracy: 0.2395 - val_loss: 3.1886 - val_categorical_accuracy: 0.1462
Epoch 5/20
193/193 [==============================] - 62s 319ms/step - loss: 2.2766 - categorical_accuracy: 0.2980 - val_loss: 3.6804 - val_categorical_accuracy: 0.1170
Epoch 6/20
193/193 [==============================] - 73s 377ms/step - loss: 2.0357 - categorical_accuracy: 0.3830 - val_loss: 4.1632 - val_categorical_ac

In [20]:
model = load_model('model_0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 128)       7

In [21]:
len(X_test)/16
accuracy = model.evaluate(X_test, Y_test)

6/6 [==============================] - 1s 108ms/step - loss: 0.4009 - categorical_accuracy: 0.8947


In [12]:
mod = tf.keras.applications.MobileNet()

In [13]:
mod.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
mod.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [ ]:
plt.imshow(X[1, :, :, 2], interpolation='nearest')

In [ ]:
print(Y[1])

In [4]:
import numpy as np
a = [[2,4,9], [2,1,4], [1,1,2]]
b = [[1,2,3], [-4,7,4], [2,-5,1]]
print(np.matmul(np.array(a), np.array(b)))

[[  4 -13  31]
 [  6  -9  14]
 [  1  -1   9]]


In [26]:
pip show tensorflow

Name: tensorflow
Version: 2.4.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/sneha/opt/anaconda3/lib/python3.8/site-packages
Requires: opt-einsum, absl-py, gast, termcolor, google-pasta, protobuf, flatbuffers, h5py, wrapt, tensorboard, grpcio, tensorflow-estimator, six, astunparse, typing-extensions, wheel, numpy, keras-preprocessing
Required-by: 
Note: you may need to restart the kernel to use updated packages.
